In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import warnings
import math
import sys
import holidays

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn import metrics

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from math import sqrt


from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import WDL,xDeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names



In [8]:
dataframe = pd.read_csv('../train.csv', sep=',', nrows=100000)
destinations = pd.read_csv('../destinations.csv', sep=',')
dataframe.shape

(100000, 24)

In [9]:
dataframe = dataframe.rename(columns={'hotel_cluster': 'item_id', 'is_booking': 'rating'})

In [11]:
len(dataframe['user_location_country'].unique())

123

In [14]:
info_dic = {'date_time':'Timestamp',
'site_name':'ID of the Expedia point of sale (i.e. Expedia.com, Expedia.co.uk, Expedia.co.jp, …)',
'posa_continent':'ID of continent associated with site_name',
'user_location_country':'The ID of the country the customer is located',
'user_location_region':'The ID of the region the customer is located',
'user_location_city':'The ID of the city the customer is located',
'orig_destination_distance':'Physical distance between a hotel and a customer at the time of search. A null means the distance could not be calculated',
'user_id':'ID of user','is_mobile':'1 when a user connected from a mobile device, 0 otherwise',
'is_package':'1 if the click/booking was generated as a part of a package (i.e. combined with a flight), 0 otherwise',
'channel':'ID of a marketing channel',
'srch_ci':'Checkin date','srch_co':'Checkout date',
'srch_adults_cnt':'The number of adults specified in the hotel room',
'srch_children_cnt':'The number of (extra occupancy) children specified in the hotel room',
'srch_rm_cnt':'The number of hotel rooms specified in the search',
'srch_destination_id':'ID of the destination where the hotel search was performed',
'srch_destination_type_id':'Type of destination',
'hotel_continent':'Hotel continent',
'hotel_country':'Hotel country',
'hotel_market':'Hotel market','rating':'(is_booking)1 if a booking, 0 if a click',
'cnt':'Numer of similar events in the context of the same user session','item_id':'(hotel_cluster)ID of a hotel cluster'}

In [15]:
info_dic['srch_ci'],info_dic['srch_co']

('Checkin date', 'Checkout date')

In [16]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date_time                  100000 non-null  object 
 1   site_name                  100000 non-null  int64  
 2   posa_continent             100000 non-null  int64  
 3   user_location_country      100000 non-null  int64  
 4   user_location_region       100000 non-null  int64  
 5   user_location_city         100000 non-null  int64  
 6   orig_destination_distance  63078 non-null   float64
 7   user_id                    100000 non-null  int64  
 8   is_mobile                  100000 non-null  int64  
 9   is_package                 100000 non-null  int64  
 10  channel                    100000 non-null  int64  
 11  srch_ci                    99929 non-null   object 
 12  srch_co                    99929 non-null   object 
 13  srch_adults_cnt            100

# Feature Engineering


In [17]:
dataframe= dataframe.drop(['orig_destination_distance'],axis=1)
dataframe = dataframe.dropna()

In [18]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99929 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   date_time                 99929 non-null  object
 1   site_name                 99929 non-null  int64 
 2   posa_continent            99929 non-null  int64 
 3   user_location_country     99929 non-null  int64 
 4   user_location_region      99929 non-null  int64 
 5   user_location_city        99929 non-null  int64 
 6   user_id                   99929 non-null  int64 
 7   is_mobile                 99929 non-null  int64 
 8   is_package                99929 non-null  int64 
 9   channel                   99929 non-null  int64 
 10  srch_ci                   99929 non-null  object
 11  srch_co                   99929 non-null  object
 12  srch_adults_cnt           99929 non-null  int64 
 13  srch_children_cnt         99929 non-null  int64 
 14  srch_rm_cnt           

In [19]:
ca_holidays = holidays.Canada()
us_holidays = holidays.UnitedStates()

uk_holidays = holidays.UnitedKingdom()
gr_holidays = holidays.Germany()

In [20]:
# check if checkin or checkout date is in holiday of different countries
dataframe['north_am_ci'] = dataframe['srch_ci'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
dataframe['north_am_co'] = dataframe['srch_co'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)

dataframe['europe_ci'] = dataframe['srch_ci'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)
dataframe['europe_co'] = dataframe['srch_co'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)

In [22]:
dataframe.head(n=2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,user_id,is_mobile,is_package,channel,...,rating,cnt,hotel_continent,hotel_country,hotel_market,item_id,north_am_ci,north_am_co,europe_ci,europe_co
0,2014-08-11 07:46:59,2,3,66,348,48862,12,0,1,9,...,0,3,2,50,628,1,0,0,0,0
1,2014-08-11 08:22:12,2,3,66,348,48862,12,0,1,9,...,1,1,2,50,628,1,0,0,0,0


In [24]:
# extract month from date_time
dataframe['click_month'] = pd.DatetimeIndex(dataframe['date_time']).month
dataframe['checkin_month'] = pd.DatetimeIndex(dataframe['srch_ci']).month
dataframe['checkout_month'] = pd.DatetimeIndex(dataframe['srch_co']).month

In [25]:
dataframe= dataframe.drop(['date_time'],axis=1)
dataframe= dataframe.drop(['srch_ci'],axis=1)
dataframe= dataframe.drop(['srch_co'],axis=1)

In [26]:
dataframe.shape

(99929, 27)

In [27]:
dataframe.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city', 'user_id', 'is_mobile',
       'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'rating', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'item_id', 'north_am_ci', 'north_am_co', 'europe_ci', 'europe_co',
       'click_month', 'checkin_month', 'checkout_month'],
      dtype='object')

In [28]:
# categ_sparse / conti_dense
sparse_features = ['site_name','posa_continent','user_location_country','user_location_region','user_location_city',
             'user_id','is_mobile','is_package','channel','click_month','checkin_month','checkout_month',
            'srch_adults_cnt','srch_children_cnt','srch_rm_cnt','srch_destination_id','hotel_continent',
               'hotel_country','cnt','north_am_ci', 'north_am_co', 'europe_ci', 'europe_co','click_month',
                   'checkin_month','checkout_month']

dense_features = ['hotel_market']
target = ['rating']

## Label Encoding for Sparse Features and Normalization For Dense Features


In [29]:
# Label Encoding for sparse features,and normalization for dense numerical features
for feat in sparse_features:
    lbe = LabelEncoder()
    dataframe[feat] = lbe.fit_transform(dataframe[feat])

In [30]:
mms = MinMaxScaler(feature_range=(0,1))
dataframe[dense_features] = mms.fit_transform(dataframe[dense_features])

### Generate feature columns
For sparse features, we transform them into dense vectors by embedding techniques. For dense numerical features, we concatenate them to the input tensors of fully connected layer.

In [31]:
# count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, dataframe[feat].nunique(),embedding_dim=4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [32]:
# generate input data for model
train, test = train_test_split(dataframe, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

### MOdel before Hyper paramter Tuning

In [37]:
model = xDeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 22s - loss: 0.2733 - binary_crossentropy: 0.2731 - val_loss: 0.2487 - val_binary_crossentropy: 0.2483
Epoch 2/10
63954/63954 - 18s - loss: 0.2229 - binary_crossentropy: 0.2221 - val_loss: 0.2652 - val_binary_crossentropy: 0.2641
Epoch 3/10
63954/63954 - 19s - loss: 0.2078 - binary_crossentropy: 0.2065 - val_loss: 0.2876 - val_binary_crossentropy: 0.2859
Epoch 4/10
63954/63954 - 19s - loss: 0.1993 - binary_crossentropy: 0.1975 - val_loss: 0.2982 - val_binary_crossentropy: 0.2961
Epoch 5/10
63954/63954 - 18s - loss: 0.1927 - binary_crossentropy: 0.1905 - val_loss: 0.3111 - val_binary_crossentropy: 0.3086
Epoch 6/10
63954/63954 - 19s - loss: 0.1885 - binary_crossentropy: 

In [38]:
auc = roc_auc_score(test[target].values, pred_ans)

print("RMSE:\t%f" % np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3),
      "MAE:\t%f" % np.round(mean_absolute_error(test[target].values, pred_ans),3),
      "MSE:\t%f" % np.round(mean_squared_error(test[target].values, pred_ans),3),
      "AUC:\t%f" % np.round(auc,3),
      sep='\n')

RMSE:	0.294000
MAE:	0.146000
MSE:	0.087000
AUC:	0.705000


### To get best parameters 


In [39]:
param_rand = {'dnn_hidden_units' : [(1,1),(2,2),(4,4),(8,8),(29,29),(128,128),(256,256)],
              'cin_layer_size':[(1,1),(2,2),(4,4),(8,8),(32,32),(128,128)],
              'cin_split_half':[True,False],
              'dnn_use_bn':[True,False],
              'l2_reg_cin':[1e-7,1e-5,1e-3,1e-1,1],
              'l2_reg_linear':[1e-5,1e-3,1e-1,1,10],
              'l2_reg_embedding':[1e-7,1e-5,1e-3,1e-1,1],
              'l2_reg_dnn':[0,0.2,2,4],
              'dnn_dropout':[0, 0.3, 0.5, 0.7, 0.9]
             }
result = {}

### To find the best parameter for DNN_Hidden_Unit

In [ ]:
dnn_hidden_units_dict = {}
for i in param_rand['dnn_hidden_units']:
    model = xDeepFM(linear_feature_columns,dnn_feature_columns, dnn_hidden_units=i, 
              seed=1024,task='binary')
    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    dnn_hidden_units_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3),
      "MAE" : np.round(mean_absolute_error(test[target].values, pred_ans),3),
      "MSE" : np.round(mean_squared_error(test[target].values, pred_ans),3),
      "AUC" : np.round(auc,3)}
result['dnn_hidden_units']=dnn_hidden_units_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 22s - loss: 0.0813 - mean_squared_error: 0.0811 - val_loss: 0.0709 - val_mean_squared_error: 0.0705
Epoch 2/10
63954/63954 - 19s - loss: 0.0666 - mean_squared_error: 0.0660 - val_loss: 0.0725 - val_mean_squared_error: 0.0717
Epoch 3/10
63954/63954 - 19s - loss: 0.0633 - mean_squared_error: 0.0624 - val_loss: 0.0771 - val_mean_squared_error: 0.0761
Epoch 4/10
63954/63954 - 19s - loss: 0.0611 - mean_squared_error: 0.0599 - val_loss: 0.0790 - val_mean_squared_error: 0.0778
Epoch 5/10
63954/63954 - 19s - loss: 0.0594 - mean_squared_error: 0.0581 - val_loss: 0.0814 - val_mean_squared_error: 0.0801
Epoch 6/10
63954/63954 - 20s - loss: 0.0579 - mean_squared_error: 0.0565 - val_loss: 0.0808 - val_mean_squared_error: 0.0794
Epoch 7/10
63954/63954 - 20s - loss: 0.0567 - mean_squared_error: 0.0553 - val_loss: 0.0835 - val_mean_squared_error: 0.0821
Epoch 8/10
63954/63954 - 20s - loss: 0.0557 - mean_squared_error: 0.0543 - 